In [1]:
import numpy as np
from scipy.optimize import fsolve
import pandas as pd
import os
from datetime import datetime

# Aerogel (6 mm) 
To simulate aerogels of 2, 4, 8, or 12 mm, change the value of "t_ag" in Input Parameters to 0.002, 0.004, 0.008, or 0.012.

In [24]:
# --- Input Parameters ---
params = {
    "q_sun": 1000,           # Incident solar flux (W/m^2), adjusted for aerogel's transmittance and absorber's reflection  
    "ref_abs": 0.9645,       # Solar absorber's reflectance (3.55%)    
    "condition": 'indoor',  
    "T_amb_c": 25,           # Ambient air temperature (Celsius)
    "T_b_c": 30,             # Back wall temperature (Celsius), set to be 5C higher than ambient
    "v": 0.1,                # Wind speed (m/s)
    "a": 0.05,               # Device width (m)
    "b": 0.0035,             # Unit stage thickness (m)
    "t_ins": 0.01,           # Insulation thickness (m)
    "t_ag": 0.006,           # Aerogel thickness (m)
    "D_a": 3.0e-5,           # Diffusivity of vapor in air (m²/s)
    "h_a": 10.3,             # Convective heat transfer coefficient of air, upward (W/m²K)
    "k": 0.04,               # Thermal conductivity of insulation (W/mK)
    "k_air": 0.026,          # Thermal conductivity of air (W/mK)
    "k_ag": 0.022,           # Thermal conductivity of aerogel (W/mK)
    "Pr": 0.71,              # Prandtl number for air 
    "epsilon": 0.95,         # Emissivity of the solar absorber (CNT: 0.95, SSA: 0.03)
    "h_fg": 2357e3,          # Latent heat of vaporization (J/kg, converted from kJ/kg)
    "M_water": 0.018015,     # Molar mass of water (kg/mol)
    "R_gas": 8.314,          # Universal gas constant (J/(mol*K))
    "sigma": 5.67e-8         # Stefan-Boltzmann constant (W/m²K⁴)
}

In [26]:
# Obtain convective heat transfer coefficient (h_a)    
def get_air_properties(T_amb_c):
    T_amb_k = T_amb_c + 273.15
     # Density using ideal gas law at 1 atm (101325 Pa)
    P_atm = 101325  
    R_air = 287.05  # J/(kg·K) - specific gas constant for air
    rho = P_atm / (R_air * T_amb_k)  # kg/m³

    # Dynamic viscosity using Sutherland's formula
    T_ref = 273.15
    mu_ref = 1.716e-5  # Pa·s at T_ref       
    S = 110.4  # K (Sutherland constant for air)
    mu = mu_ref * (T_amb_k / T_ref)**1.5 * (T_ref + S) / (T_amb_k + S)  # Pa·s
    
    return rho, mu
    
def get_h_a_windy(T_amb_c, v, a):
    T_amb_k = T_amb_c + 273.15
    
    rho, mu = get_air_properties(T_amb_c)
      
    # Calculate Re at length L (device width)
    Re_L = rho * v * a / mu
    if Re_L >= 5e5:
        print(f"Warning: Reynolds number ({Re_L:.2e}) exceeds laminar flow limit (5e5).")
        print(f"The correlation may not be accurate for turbulent flow.")

    # Calculate average convection coefficient 
    h_a = 2 * 0.453 * params["Pr"]**(1/3) * (rho * v / mu)**0.5 * params["k_air"] * a**(-0.5)

    return h_a

def get_h_a_windless(T_amb_c, a):
    T_amb_k = T_amb_c + 273.15
    T_abs_k = 273.15 + 60              # assume 60C for the solar absorber 
    T_avg_k = (T_amb_k + T_abs_k) / 2  # Film temperature for properties

    rho, mu = get_air_properties(T_avg_k - 273.15)

    # Calculate kinematic viscosity
    nu = mu / rho  # m²/s

    # Gravitational acceleration
    g = 9.81  # m/s²

    # Coefficient of volume expansion for ideal gas: β = 1/T
    beta = 1 / T_avg_k  # 1/K
    
    # Characteristic length for horizontal plate
    l_c = a**2/(4*a)
    
    # Calculate Rayleigh number 
    Ra = (g * beta * abs(T_abs_k - T_amb_k) * l_c**3 * params["Pr"]) / (nu**2)

    # Check if Ra is in valid range
    if Ra < 1e4:
        print(f"Warning: Rayleigh number ({Ra:.2e}) is below the correlation range (10^4 to 10^7).")
        print(f"Natural convection may be negligible.")
    elif Ra > 1e7:
        print(f"Warning: Rayleigh number ({Ra:.2e}) exceeds the correlation range (10^4 to 10^7).")
        print(f"The correlation may not be accurate for turbulent natural convection.")

    # Calculate natural convection coefficient (Nu = h_a * k_air/l_c = 0.54 * Ra^0.25)
    h_a = 0.54 * Ra**0.25 * params["k_air"] / l_c

    return h_a

# Obtain measured solar weighted transmittance from aerogel thickness 
def get_Tr_from_thickness(t_ag):
    thickness_data = np.array([0.002, 0.004, 0.006, 0.008, 0.012])
    Tr_data = np.array([0.9758, 0.9640, 0.9463, 0.9285, 0.9155])
    return np.interp(t_ag, thickness_data, Tr_data)
    
# Update parameters of transmittance and convective heat transfer coefficient  
params["Tr"] = get_Tr_from_thickness(params["t_ag"])


In [57]:
# --- Load and process spectral data ---
try:
    file_path = os.path.join("data", "AG_transmittance.csv")
    
    column_names = ['wavelength_m', 'AG2', 'AG4', 'AG6', 'AG8', 'AG12', 'atm']
    AG_data = pd.read_csv(file_path, skiprows=1, names=column_names)

    for col in column_names:
        AG_data[col] = pd.to_numeric(AG_data[col])

    # 3) Calculate the difference between adjacent wavelengths (delta_lambda)
    AG_data['delta_lambda'] = AG_data['wavelength_m'].diff().bfill()

    print("Successfully loaded and processed Aerogel and atmospheric spectral data.")
    
except Exception as e:
    print(f"Error loading or processing AG_transmittance.csv: {e}")
    AG_data = None

Successfully loaded and processed Aerogel and atmospheric spectral data.


In [28]:
# --- Helper Functions ---
def planck_law(wav, T):
    """Calculates spectral radiance using Planck's Law."""
    C1 = 3.7418e-16
    C2 = 1.4388e-2
    return (C1 / (wav**5)) / (np.exp(C2 / (wav * T)) - 1)

def calculate_q_rad(T_f_k, p, spectral_data):
    """
    Calculates q_rad by integrating Planck's law over the spectral data.
    """
    if spectral_data is None: return 0
    T_amb_k = p["T_amb_c"] + 273.15
    
    E_f = planck_law(spectral_data['wavelength_m'], T_f_k)
    E_amb = planck_law(spectral_data['wavelength_m'], T_amb_k)
    
    if p["t_ag"] == 0.002:
        tr = spectral_data['AG2']
    if p["t_ag"] == 0.004:
        tr = spectral_data['AG4']
    if p["t_ag"] == 0.006:
        tr = spectral_data['AG6']
    if p["t_ag"] == 0.008:
        tr = spectral_data['AG8']
    if p["t_ag"] == 0.012:
        tr = spectral_data['AG12']
        
    delta_lambda = spectral_data['delta_lambda']
    epsilon = p["epsilon"]

    if p["condition"] == 'indoor':
        integrand = tr * epsilon * (E_f - E_amb) * delta_lambda
        q_rad = integrand.sum()
    else: # outdoor
        trans_atm = spectral_data['atm']
        integrand = tr * epsilon * (E_f - (1 - trans_atm) * E_amb) * delta_lambda
        q_rad = integrand.sum()       
    return q_rad

def calculate_saturation_concentration(T_k, activity=1.0):
    """
    Calculates water vapor saturation concentration (mol/m³) at a given temperature,
    using August-Roche-Magnus formula, adjusted for the solution's activity.
    """
    T_c = T_k - 273.15
    # P_sat_pure = 610.94 * np.exp((17.625 * T_c) / (T_c + 243.04)) # August-Roche-Magnus formula
    P_sat_pure = 133.32 * 10**8.07131 / 10**(1730.63/(T_c + 233.426)) # Antoine equation
    P_sat_solution = P_sat_pure * activity
    return P_sat_solution / (params["R_gas"] * T_k)

def find_top_surface_temp(T_f_k, p):
    T_amb_k = p["T_amb_c"] + 273.15
    def top_surface_balance(T_top_k):
        q_cond_up = p["k_ag"] * (T_f_k - T_top_k) / p["t_ag"]
        q_conv_out = p["h_a"] * (T_top_k - T_amb_k)
        if p["condition"] == 'indoor':
            q_rad_out = p["epsilon"] * p["sigma"] * (T_top_k**4 - T_amb_k**4)
        if p["condition"] == 'outdoor':
            q_rad_out = (0.0348*(T_top_k-273.15)**2 + 3.75*(T_top_k-273.15) + 113.2) 
        return q_cond_up - (q_conv_out + q_rad_out)
    initial_guess_T_top = (T_f_k + T_amb_k) / 2
    return fsolve(top_surface_balance, initial_guess_T_top)[0]

def solve_stage_backward(q_out, T_b_c, p, is_first_stage=False):
    """
    Solves a single stage backward to find the input heat and front wall temp.
    """
    T_b_k = T_b_c + 273.15

    def find_Tf_from_q_out(T_f_c, q_out_target, T_b_k, p):
        T_f_k = T_f_c + 273.15
        q_cond = p["k_air"] * (T_f_k - T_b_k) / p["b"]
        q_rad_int = 0.95 * p["sigma"]*(T_f_k**4 - T_b_k**4)
        # Use activity=0.98 for evaporating seawater at the front wall
        c_f = calculate_saturation_concentration(T_f_k, activity=0.98)
        # Use activity=1.0 for condensing pure water at the back wall
        c_b = calculate_saturation_concentration(T_b_k, activity=1.0)
        J_evap_molar = p["D_a"] * (c_f - c_b) / p["b"]
        q_evap = (J_evap_molar * p["M_water"]) * p["h_fg"]
        return (q_cond + q_evap + q_rad_int) - q_out_target

    initial_guess_T_f = T_b_c + 5.0
    T_f_c_solved = fsolve(find_Tf_from_q_out, initial_guess_T_f, args=(q_out, T_b_k, p))[0]
    T_f_k_solved = T_f_c_solved + 273.15

    T_top_k_solved = find_top_surface_temp(T_f_k_solved, p)
    T_top_c_solved = T_top_k_solved - 273.15

    T_amb_k = p["T_amb_c"] + 273.15
    R_side = (1 /p["h_a"]) + (p["t_ins"] / p["k"])
    T_avg_side_k = (T_f_k_solved + T_b_k) / 2
    q_side_flux = (T_avg_side_k - T_amb_k) / R_side
    q_side_per_front_area = q_side_flux * (4 * p["b"] / p["a"])

    q_in = q_out + q_side_per_front_area

    if is_first_stage:
        q_rad = calculate_q_rad(T_f_k_solved, p, AG_data)

        R_top = (1 / p["h_a"]) + (p["t_ag"] / p["k_ag"])
        q_cond_up = (T_f_k_solved - T_amb_k) / R_top
        R_top_side = (1 / p["h_a"]) 
        q_cond_side = (((T_f_k_solved + T_top_k_solved) / 2) - T_amb_k) / R_side * (4 * p["t_ag"] / p["a"])

            
        q_in += q_rad + q_cond_up + q_cond_side

    return q_in, T_f_c_solved, T_top_c_solved

In [52]:
# --- Main Multi-Stage Simulation ---
def run_multistage_simulation(n_stages, params):

    q_out_n = params["q_sun"] * params["ref_abs"] * params["Tr"]
    tolerance = 0.1
    max_iterations = 100
    stage_data = []

    print(f"--- Running Multi-Stage Simulation for {n_stages} Stages ---\n")

    for iteration in range(max_iterations):
        current_stage_data = []
        q_out_current = q_out_n
        T_b_current_c = params["T_b_c"]

        for i in range(n_stages, 0, -1):
            is_first = (i == 1)
            q_in_current, T_f_current_c, T_top_current_c = solve_stage_backward(q_out_current, T_b_current_c, params, is_first_stage=is_first)
            current_stage_data.append({"stage": i, "T_f": T_f_current_c, "T_b": T_b_current_c, "T_top": T_top_current_c})
            q_out_current = q_in_current
            T_b_current_c = T_f_current_c

        current_stage_data.reverse()
        q_in_1_calculated = q_out_current 
        error = abs(q_in_1_calculated - params["q_sun"]* params["ref_abs"] * params["Tr"])

        print(f"Iteration {iteration + 1}: Calculated q_in,1 = {q_in_1_calculated:.2f} W/m^2, Error = {error:.2f} W/m^2")

        if error < tolerance:
            print("\nConvergence reached!")
            stage_data = current_stage_data
            break

        scaling_factor = params["q_sun"]* params["ref_abs"] * params["Tr"] / q_in_1_calculated
        q_out_n *= scaling_factor
    else:
        print("\nWarning: Maximum iterations reached without convergence.")
        stage_data = current_stage_data

    # --- Final Calculations and Output ---
    print("\n--- Per-Stage Detailed Results ---")
    total_evap_heat = 0
    for data in stage_data:
        T_f_c = data["T_f"]
        T_b_c = data["T_b"]
        T_f_k = T_f_c + 273.15
        T_b_k = T_b_c + 273.15

        # Calculate vapor flux for this stage 
        c_f = calculate_saturation_concentration(T_f_k, activity=0.98)
        c_b = calculate_saturation_concentration(T_b_k, activity=1.0)
        J_evap_molar = params["D_a"] * (c_f - c_b) / params["b"]
        J_evap_mass = J_evap_molar * params["M_water"]
        J_evap_kg_hr = J_evap_mass * 3600
        
        # Calculate heat carried by evaporation
        q_evap = J_evap_mass * params["h_fg"]
        total_evap_heat += q_evap

        if data['stage'] == 1:
            T_top_c = data["T_top"]
            top_temp_str = f"| Top Surface Temp: {T_top_c:.2f}°C "
        else: top_temp_str = ""

        print(f"  Stage {data['stage']}: Evap Temp: {data['T_f']:.2f}°C | Condensation Temp: {data['T_b']:.2f}°C | Vapor Flux: {J_evap_kg_hr:.2f} kg/m²/hr")

    print(f"\nTop Surface Temp: { T_top_c:.2f}°C ")
    print("\n--- Overall Performance ---")
    eta_tot = total_evap_heat / params["q_sun"] 
    total_vapor_flux_kg_hr = (total_evap_heat / params["h_fg"]) * 3600
    print(f"Total Vapor Mass Flux: {total_vapor_flux_kg_hr:.2f} kg/m²/hr")
    print(f"Total Efficiency (eta_tot): {eta_tot * 100:.2f}%")

    results = {
        "water productivity (kg/m2/hr)": round(total_vapor_flux_kg_hr, 2),
        "total efficiency (%)": round(eta_tot * 100, 2),
        "absorber temp (C)": round(stage_data[0]['T_f'], 2) if stage_data else None,
        "top surface temp (C)": round(stage_data[0]['T_top'], 2) if stage_data else None,
    }
    return results

# Simulation Cases in Figure 3 

In [53]:
simulation_cases_Fig3 = [
    {
        "name": "Indoor, windless", #Figure 3b
        "condition": 'indoor',
        "v": 0,
        "a": 0.05,
    },    
    {
        "name": "Indoor, 2 m/s wind", #Figure 3b
        "condition": 'indoor',
        "v": 2.0,
    },
    {
        "name": "Indoor, 4 m/s wind", #Figure 3b
        "condition": 'indoor',
        "v": 4.0,
    },
    {
        "name": "Indoor, 6 m/s wind", #Figure 3b
        "condition": 'indoor',
        "v": 6.0,
    },   
    {
        "name": "Outdoor, windless", #Figure 3c
        "condition": "outdoor",
        "v": 0,
    },       
    {
        "name": "Outdoor, 2 m/s wind", #Figure 3c
        "condition": "outdoor",
        "v": 2.0,
    },      
    {
        "name": "Outdoor, windless, 20C ambient temperature", #Figure 3e
        "condition": "outdoor",
        "T_b_c": 25,
        "T_amb_c": 20,
        "v": 0, 
    },
    {
        "name": "Outdoor, windless, 30C ambient temperature", #Figure 3e
        "condition": "outdoor",
        "T_b_c": 35,
        "T_amb_c": 30,
        "v": 0, 
    },
    {
        "name": "Outdoor, windless, 35C ambient temperature", #Figure 3e
        "condition": "outdoor",
        "T_b_c": 40,
        "T_amb_c": 35,
        "v": 0,
    },
    {
        "name": "Outdoor, windless, 40C ambient temperature", #Figure 3e
        "condition": "outdoor",
        "T_b_c": 45,
        "T_amb_c": 40,
        "v": 0, 
    },    
    {
        "name": "Outdoor, 0.7 kW/m2 solar flux", #Figure 3f
        "condition": "outdoor",
        "q_sun": 700, 
        "T_b_c": 30,
        "T_amb_c": 25,
        "v": 0,
    },
    {
        "name": "Outdoor, 0.5 kW/m2 solar flux", #Figure 3f
        "condition": "outdoor",
        "q_sun": 500, 
        "T_b_c": 30,
        "T_amb_c": 25,
        "v": 0,
    }, 
    {
        "name": "Outdoor, 0.3 kW/m2 solar flux", #Figure 3f
        "condition": "outdoor",
        "q_sun": 300, 
        "T_b_c": 30,
        "T_amb_c": 25,
        "v": 0,
    },    
]

In [58]:
# --- Run the Simulation to Reproduce Figure 3 ---
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

if __name__ == "__main__":
    number_of_stages =  6

    all_results_data = []

    for case in simulation_cases_Fig3:
        print(f"\n========================================================")
        print(f"Running: {case['name']}")
        print(f"========================================================")

        current_params = params.copy()
        current_params.update(case)

        if current_params["v"] <= 0.1:  # Windless
            current_params["h_a"] = get_h_a_windless(current_params["T_amb_c"], current_params["a"])
        else:  # Windy
            current_params["h_a"] = get_h_a_windy(current_params["T_amb_c"], current_params["v"], current_params["a"])
            
        current_params["Tr"] = get_Tr_from_thickness(current_params["t_ag"])

        output_results = run_multistage_simulation(number_of_stages, current_params)

        result_row = {"name": case["name"]}
        result_row.update(output_results)
        
        all_results_data.append(result_row)
        

    results_df = pd.DataFrame(all_results_data)

    output_filename = os.path.join("results", f"{params["t_ag"]*1000:.0f}mmAerogel_simulation_Fig3_summary_{timestamp}.csv")
    results_df.to_csv(output_filename, index=False)

    print(f"\n\n All simulations complete. Results saved to '{output_filename}'")


Running: Indoor, windless
Natural convection may be negligible.
--- Running Multi-Stage Simulation for 6 Stages ---

Iteration 1: Calculated q_in,1 = 1311.22 W/m^2, Error = 398.52 W/m^2
Iteration 2: Calculated q_in,1 = 968.67 W/m^2, Error = 55.96 W/m^2
Iteration 3: Calculated q_in,1 = 922.11 W/m^2, Error = 9.41 W/m^2
Iteration 4: Calculated q_in,1 = 914.33 W/m^2, Error = 1.63 W/m^2
Iteration 5: Calculated q_in,1 = 912.99 W/m^2, Error = 0.28 W/m^2
Iteration 6: Calculated q_in,1 = 912.76 W/m^2, Error = 0.05 W/m^2

Convergence reached!

--- Per-Stage Detailed Results ---
  Stage 1: Evap Temp: 70.88°C | Condensation Temp: 66.28°C | Vapor Flux: 0.95 kg/m²/hr
  Stage 2: Evap Temp: 66.28°C | Condensation Temp: 61.24°C | Vapor Flux: 0.89 kg/m²/hr
  Stage 3: Evap Temp: 61.24°C | Condensation Temp: 55.57°C | Vapor Flux: 0.84 kg/m²/hr
  Stage 4: Evap Temp: 55.57°C | Condensation Temp: 48.97°C | Vapor Flux: 0.79 kg/m²/hr
  Stage 5: Evap Temp: 48.97°C | Condensation Temp: 40.87°C | Vapor Flux: 0.7